# Data Exploration

# 1.Path

In [ ]:
from ipywidgets import interactive
import os
MAIN_DIR = 'sources'

def getSubFolder(subfoldername):
    return os.path.join(w.result, subfoldername).replace("\\","/")


def getPath(filename):
    result = os.path.join(MAIN_DIR, filename).replace("\\","/")
    global w2
    w2 = interactive(getSubFolder,subfoldername=os.listdir(result))
    return result
    
w = interactive(getPath,filename=os.listdir(MAIN_DIR))


In [ ]:
display(w)

In [ ]:
display(w2)

In [ ]:
w2.result

In [ ]:
path = w2.result

import os

directory_contents = os.listdir(path)
beaver_id = []

for n in range(0,len(directory_contents)):
    beaver_id.append(directory_contents[n])

# Sorting list of Integers in ascending
beaver_id.sort(key=int)

print(beaver_id)

The data set contains 10 classes.

# 2. Images Counter

In [ ]:
import glob
jpgCounter = []
totalDir = 0

for base, dirs, files in os.walk(path):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    jpgCount = len(glob.glob1(base,"*.jpg"))
    jpgCounter.append(jpgCount)
    print('number of jpg :', jpgCount)
del jpgCounter[0] # delete first index

print('Total Number of directories',totalDir)
print('Total Number of jpg',sum(jpgCounter))

# 3. Graph Data Counter

In [ ]:
import matplotlib.pyplot as plt

# Create a List of Labels for x-axis
names = beaver_id

# Create a List of Values (Same Length as Names List)
samples = jpgCounter

# Make the Chart
plt.bar(names, samples)

# Title
plt.title('Number of Images per Class')

# X and y labels
plt.xlabel("Beaver")
plt.ylabel("Count of images")

# Show the Chart
plt.show()

Here are some of beaver photos from dataset

# 4. Image Visualization

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread
import os
import random


def plot_image(path,n):
    
    image_list = []

    # read the entries
    with os.scandir(path) as listOfEntries:
        for entry in listOfEntries:
            # print all entries that are files
            if entry.is_file():
                #print(entry.name)
                image_list.append(entry.name)


    # filter only jpg or png files
    image_list = list(filter(lambda k: ('jpg' or 'png') in k, image_list))
    image_list.sort()
    pyplot.figure(figsize=(10, 10))
    # plot all images
    for i in range(n):
        random_num = random.randint(0, len(image_list)-1)
        # define subplot
        pyplot.subplot(2,3, i+1)
        # define filename
        filename = path + '/' + image_list[random_num]
        # show image name
        pyplot.title(image_list[random_num])
        # load image pixels
        image = imread(filename)
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    pyplot.show()

Here are some of random picture of beaver 1:

In [ ]:
print('Beaver 1')
plot_image(path+'/1',2)

Here are some of random picture of beaver 2:

In [ ]:
print('Beaver 2')
plot_image(path+'/2',2)

Here are some of random picture of beaver 3:

In [ ]:
print('Beaver 3')
plot_image(path+'/3',2)

# 5. Create a dataset

It's good practice to use a validation split when testing the algorihtm. Let's use 80% of the images for data base, and 20% for testing.

Create a dataset directories of train and test

In [ ]:
# importing os module
import os
# create subdirectories
subdirs = ['data_base/', 'test/']
data_path = path.split("/")
print(data_path)
for subdir in subdirs:
    # create label subdirectories
    for labldir in beaver_id:
        newdir = os.path.join(data_path[0],data_path[1], subdir,labldir)
        os.makedirs(newdir, exist_ok=True)

Move each files randomly to train or test directories

In [ ]:
import random
from shutil import copyfile
from os import listdir

# seed random number generator
random.seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.70
# copy training dataset images into subdirectories
src_directory = path
for i, file in enumerate(listdir(src_directory)):
    src = src_directory + '/' + file

    for subfile in listdir(src):
        dst_dir = 'data_base/'
        if random.random() < val_ratio:
            dst_dir = 'test/'
    
        dst = os.path.join(data_path[0],data_path[1], dst_dir, file, subfile)
        copyfile(os.path.join(src, subfile), dst)

**Double check if if you have at least 1 picture in data_base and test folders!**

# 6. Prediction using SIFT algorthim

## 6.1 Importing Libraries

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot

# Single test images

In [ ]:
# Iterate all the folder inside the data base

y_pred = []
y_true = []

id_prediction = []
match_lines = []
image_name = []

for labldir in beaver_id:
    src = os.path.join(data_path[0],data_path[1], 'data_base',str(labldir))

    for file_name in listdir(src):
        print(os.path.join(src, file_name))

        fingerprint_database_image = cv2.imread(os.path.join("/Users/gagad/Desktop/beaver/",src, file_name))
        fingerprint_database_image = cv2.resize(fingerprint_database_image,(640,480))
        sift = cv2.xfeatures2d.SIFT_create()

        # load image pixels
        test_image = cv2.imread("/Users/gagad/Desktop/beaver/sources/sample1/original/1/7.jpg")
        test_image = cv2.resize(test_image,(640,480))

        keypoints_1, descriptors_1 = sift.detectAndCompute(test_image, None)
        keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None)


        matches = cv2.FlannBasedMatcher(dict(algorithm=2, trees=10), 
                dict()).knnMatch(descriptors_1, descriptors_2, k=2)

        match_points = []
        
        for p, q in matches:
            if p.distance < 0.75*q.distance:
                match_points.append(p)

        keypoints = 0
        if len(keypoints_1) <= len(keypoints_2):
            keypoints = len(keypoints_1)            
        else:
            keypoints = len(keypoints_2)

        print(len(match_points))
        print("% match: ", len(match_points) / keypoints * 100)
        print("Figerprint ID: " + str(file_name)) 

        id_prediction = np.append(id_prediction,int(labldir))
        match_lines = np.append(match_lines, len(match_points))
        image_name = np.append(image_name,file_name)


sorted_rank = sorted(zip(match_lines, id_prediction, image_name), reverse=True)[:3]
y_pred = np.append(y_pred, int(id_prediction[0]))

print(sorted_rank)
print("y_pred : ",y_pred)

## 6.2 Multiple test images

using number of lines

In [ ]:
# Iterate all the folder inside the data base

y_pred = []
y_true = []



for labldir in beaver_id:
    src_test = os.path.join(data_path[0],data_path[1], 'test',str(labldir))
    label = labldir
    for file_name_test in listdir(src_test):
        print(os.path.join(src_test, file_name_test))
        print("label: {}".format(label))

        y_true = np.append(y_true, int(label))

        id_prediction = []
        match_lines = []
        image_name = []
            
        for labldir in beaver_id:
            src = os.path.join(data_path[0],data_path[1], 'data_base',str(labldir))

            for file_name in listdir(src):
                print(os.path.join(src, file_name))

                fingerprint_database_image = cv2.imread(os.path.join("/Users/gagad/Desktop/beaver/",src, file_name))
                fingerprint_database_image = cv2.resize(fingerprint_database_image,(640,480))
                sift = cv2.xfeatures2d.SIFT_create()

                # load image pixels
                test_image = cv2.imread(os.path.join("/Users/gagad/Desktop/beaver/",src_test, file_name_test))
                test_image = cv2.resize(test_image,(640,480))

                keypoints_1, descriptors_1 = sift.detectAndCompute(test_image, None)
                keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None)


                matches = cv2.FlannBasedMatcher(dict(algorithm=0, trees=5), 
                        dict()).knnMatch(descriptors_1, descriptors_2, k=2)

                match_points = []
                
                for p, q in matches:
                    if p.distance < 0.75*q.distance:
                        match_points.append(p)

                keypoints = 0
                if len(keypoints_1) <= len(keypoints_2):
                    keypoints = len(keypoints_1)            
                else:
                    keypoints = len(keypoints_2)

                print(len(match_points))
                print("% match: ", len(match_points) / keypoints * 100)
                print("Figerprint ID: " + str(file_name)) 

                id_prediction = np.append(id_prediction, int(labldir))
                match_lines = np.append(match_lines, len(match_points))
                image_name = np.append(image_name, file_name)

                '''
                result = cv2.drawMatches(test_image, keypoints_1, fingerprint_database_image, 
                        keypoints_2, match_points, None) 
                result = cv2.resize(result, None, fx=2.5, fy=2.5)

    

                cv2.imshow("result",cv2.resize(result,(640,480)))
                cv2.waitKey(0)
                cv2.destroyAllWindows()

                '''
                
                


        sorted_rank = sorted(zip(match_lines, id_prediction, image_name), reverse=True)[:3]
        y_pred = np.append(y_pred, int(sorted_rank[0][1]))



        
        print(sorted_rank)
        #print('match_percentage :', match_percentage)
        #print('id_prediction :', id_prediction)
        #print('image_name :', image_name)
        print('y_pred :', y_pred)
        print('y_true :', y_true)

        print('-----------------------------------------------------------------')

In [ ]:
src_test

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = [int(i) for i in y_true]
y_pred = [int(i) for i in y_pred]
print('y_true :', y_true)
print('y_pred :', y_pred)

beaver_id = [int(i) for i in beaver_id]

cf_matrix = confusion_matrix(y_true, y_pred, labels = beaver_id )
print(cf_matrix)

np.save('cf_matrix.npy',cf_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax_sub = plt.subplots(figsize=(20,20)) 
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', ax=ax_sub)
ax.set_title('Beaver Tail Pattern Matching\n\n')
ax.set_ylabel('Predicted Beaver')
ax.set_xlabel('Actual Beaver ')
# ax.figure(figsize=(20,20))
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(beaver_id)
ax.yaxis.set_ticklabels(beaver_id)
## Display the visualization of the Confusion Matrix.
plt.show()

resize

In [ ]:
import cv2
 
img = cv2.imread('/home/com4/beaver/sources/sift/beaver1_test.jpg', cv2.IMREAD_UNCHANGED)
 
print('Original Dimensions : ',img.shape)

height = 640
width = int(img.shape[1]*height/img.shape[0])

dim = (width, height)
 
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 
cv2.imshow("Resized image", resized)
cv2.waitKey(0)

In [ ]:
import cv2

def resize_image(image):
    height = 640
    width = int(image.shape[1]*height/image.shape[0])

    dim = (width, height)
    
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return resized

In [ ]:
# Iterate all the folder inside the data base

y_pred = []
y_true = []



for labldir in beaver_id:
    src_test = os.path.join(data_path[0],data_path[1], 'test',str(labldir))

    for file_name_test in listdir(src_test):
        print(os.path.join(src_test, file_name_test))

        y_true = np.append(y_true, int(labldir))

        id_prediction = []
        match_lines = []
        image_name = []
            
        for labldir in beaver_id:
            src = os.path.join(data_path[0],data_path[1], 'data_base',str(labldir))

            for file_name in listdir(src):
                print(os.path.join(src, file_name))

                fingerprint_database_image = cv2.imread(os.path.join("/home/com4/beaver/",src, file_name))
                fingerprint_database_image = resize_image(fingerprint_database_image)
                sift = cv2.xfeatures2d.SIFT_create()

                # load image pixels
                test_image = cv2.imread(os.path.join("/home/com4/beaver/",src_test, file_name_test))
                test_image = resize_image(test_image)

                keypoints_1, descriptors_1 = sift.detectAndCompute(test_image, None)
                keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None)


                matches = cv2.FlannBasedMatcher(dict(algorithm=0, trees=5), 
                        dict()).knnMatch(descriptors_1, descriptors_2, k=2)

                match_points = []
                
                for p, q in matches:
                    if p.distance < 0.75*q.distance:
                        match_points.append(p)

                keypoints = 0
                if len(keypoints_1) <= len(keypoints_2):
                    keypoints = len(keypoints_1)            
                else:
                    keypoints = len(keypoints_2)

                print(len(match_points))
                print("% match: ", len(match_points) / keypoints * 100)
                print("Figerprint ID: " + str(file_name)) 

                id_prediction = np.append(id_prediction, int(labldir))
                match_lines = np.append(match_lines, len(match_points))
                image_name = np.append(image_name, file_name)

                '''
                result = cv2.drawMatches(test_image, keypoints_1, fingerprint_database_image, 
                        keypoints_2, match_points, None) 
                result = cv2.resize(result, None, fx=2.5, fy=2.5)

    

                cv2.imshow("result",cv2.resize(result,(640,480)))
                cv2.waitKey(0)
                cv2.destroyAllWindows()

                '''
                
                


        sorted_rank = sorted(zip(match_lines, id_prediction, image_name), reverse=True)[:3]
        y_pred = np.append(y_pred, int(sorted_rank[0][1]))



        
        print(sorted_rank)
        #print('match_percentage :', match_percentage)
        #print('id_prediction :', id_prediction)
        #print('image_name :', image_name)
        print('y_pred :', y_pred)
        print('y_true :', y_true)

        print('-----------------------------------------------------------------')